<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/140_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <a href="https://kaggle.com/kernels/welcome?src=https://github.com/gurovic/MLCourse/blob/main/140_kfold.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

# **Разделение данных и кросс-валидация**

## **Введение: Почему нельзя обучать на всех данных?**
При разработке ML-моделей критически важно правильно разделять данные, чтобы:
- ✅ **Оценить** реальную обобщающую способность модели
- 🚫 **Избежать** переобучения (overfitting)
- 📊 **Сравнить** разные модели объективно
- 🔧 **Настроить** гиперпараметры без утечки данных

**Ключевой принцип:**  
> "Тестовые данные должны быть священными — их нельзя использовать ни для обучения, ни для настройки параметров"

## **🟢 Базовый уровень: Стандартное разделение**

### **1. Простое разделение Train-Test**
Самый базовый подход — разделение на две части:
- **Train (70-80%)** — для обучения модели
- **Test (20-30%)** — для финальной оценки

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np

# Создаем синтетические данные
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,       # 20% для теста
    random_state=42,     # для воспроизводимости
    stratify=y           # сохранение распределения классов
)

print(f"Общий размер данных: {X.shape[0]}")
print(f"Train size: {X_train.shape[0]} ({X_train.shape[0]/X.shape[0]:.0%})")
print(f"Test size: {X_test.shape[0]} ({X_test.shape[0]/X.shape[0]:.0%})")

### **2. Проблемы базового подхода**
1. **Недооценка вариативности** — одно разделение может быть неудачным
2. **Потеря данных для обучения** — 20-30% не используются для обучения
3. **Смещение оценки** — особенно критично для маленьких датасетов

## **🟡 Продвинутый уровень: Валидация и кросс-валидация**

### **1. Трехстороннее разделение (Train-Validation-Test)**
Добавляем валидационный набор для настройки гиперпараметров:
- **Train (60%)** — обучение модели
- **Validation (20%)** — подбор гиперпараметров
- **Test (20%)** — финальная оценка

In [ ]:
# Первое разделение: выделяем тест
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Второе разделение: train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp  # 0.25 * 0.8 = 0.2
)

print(f"Train size: {X_train.shape[0]} ({X_train.shape[0]/X.shape[0]:.0%})")
print(f"Validation size: {X_val.shape[0]} ({X_val.shape[0]/X.shape[0]:.0%})")
print(f"Test size: {X_test.shape[0]} ({X_test.shape[0]/X.shape[0]:.0%})")

### **2. Кросс-валидация (K-Fold)**
Решает проблему потери данных и вариативности:
1. Данные делятся на K равных частей
2. Модель обучается на K-1 частях
3. Оценивается на оставшейся части
4. Процесс повторяется K раз

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []

print("Кросс-валидация (5-fold):")
for i, (train_index, val_index) in enumerate(kf.split(X)):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    scores.append(acc)
    print(f"Fold {i+1} Accuracy: {acc:.4f}")

print(f"\nСредняя Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

### **3. Стратифицированная кросс-валидация**
Для сохранения распределения классов в каждом фолде:

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

print("Стратифицированная кросс-валидация (5-fold):")
for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    scores.append(acc)
    print(f"Fold {i+1} Accuracy: {acc:.4f}")

print(f"\nСредняя Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

### **Преимущества кросс-валидации**
- **Полное использование данных** — все наблюдения участвуют в обучении и валидации
- **Надежная оценка** — уменьшение зависимости от конкретного разделения
- **Оценка стабильности** — можно измерить дисперсию качества модели

## **🔴 Экспертный уровень: Специальные случаи**

### **1. Временные ряды (Time Series Split)**
Для данных с временной зависимостью:

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd

# Создаем временной ряд
dates = pd.date_range(start="2023-01-01", periods=100, freq="D")
X = np.random.randn(100, 5)
y = np.random.randint(0, 2, 100)

tscv = TimeSeriesSplit(n_splits=5)
print("Разбиение временных рядов:")

for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {i+1}: Train size: {len(train_index)}, Test size: {len(test_index)}")
    print(f"Train dates: {dates[train_index[0]]} to {dates[train_index[-1]]}")
    print(f"Test dates: {dates[test_index[0]]} to {dates[test_index[-1]]}\n")

### **2. Групповое разделение (Group K-Fold)**
Когда данные имеют группы (пациенты, клиенты):

In [ ]:
from sklearn.model_selection import GroupKFold

# Создаем группы (например, 10 пациентов по 10 образцов)
groups = np.repeat(np.arange(10), 10)

gkf = GroupKFold(n_splits=3)
print("Групповое разделение:")

for i, (train_index, test_index) in enumerate(gkf.split(X, y, groups=groups)):
    train_groups = np.unique(groups[train_index])
    test_groups = np.unique(groups[test_index])
    print(f"Fold {i+1}: Train groups: {train_groups}, Test groups: {test_groups}")

### **3. Оставьте-одного-вне (Leave-One-Out - LOO)**
Экстремальный случай K-Fold, где K = N (число образцов):

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression

# Используем небольшой набор данных для демонстрации
X_small, y_small = X[:20], y[:20]

loo = LeaveOneOut()
scores = []

for train_index, test_index in loo.split(X_small):
    X_train, X_test = X_small[train_index], X_small[test_index]
    y_train, y_test = y_small[train_index], y_small[test_index]

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

print(f"LOO Accuracy: {np.mean(scores):.4f}")

### **4. Перекрестная валидация с подбором параметров**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1]
}

grid_search = GridSearchCV(
    SVC(),
    param_grid,
    cv=5,                   # 5-fold CV
    scoring='accuracy',
    return_train_score=True
)

grid_search.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучшая точность: {grid_search.best_score_:.4f}")

## **📊 Сравнение методов валидации**

| **Метод**               | **Когда использовать**                     | **Преимущества**                          | **Недостатки**               |
|-------------------------|------------------------------------------|-------------------------------------------|-------------------------------|
| **Train-Test Split**    | Большие датасеты, быстрые эксперименты   | Простота, скорость                        | Высокая дисперсия оценки     |
| **K-Fold CV**           | Средние датасеты, стандартные задачи     | Стабильная оценка, полное использование   | Вычислительно дорогой       |
| **Stratified K-Fold**   | Несбалансированные классы                | Сохраняет распределение классов           | Сложнее реализовать         |
| **Time Series Split**   | Временные ряды                           | Учитывает временную зависимость           | Не подходит для независимых данных|
| **Group K-Fold**        | Данные с групповой структурой            | Защищает от утечки между группами         | Требует информации о группах|

## **💡 Практические рекомендации**

### **1. Размеры выборок**
- **Маленькие датасеты (<1000 образцов):**  
  Используйте кросс-валидацию с K=5-10 или LOO
  
- **Средние датасеты (1k-100k образцов):**  
  K-Fold с K=5-10
  
- **Большие датасеты (>100k образцов):**  
  Train-Test Split (98%/1%/1% или 99%/0.5%/0.5%)

### **2. Предотвращение утечки данных**
- Всегда делайте предобработку **после** разделения

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Неправильно: утечка данных!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test = train_test_split(X_scaled, test_size=0.2)

# Правильно: использование в пайплайне
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestClassifier()
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
pipeline.fit(X_train, y_train)
score = pipeline.score(X_test, y_test)
print(f"Точность с пайплайном: {score:.4f}")

### **3. Автоматизация в sklearn**
Используйте Pipeline для предотвращения утечек:

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

pipeline = make_pipeline(
    StandardScaler(),
    PCA(n_components=0.95),
    RandomForestClassifier(random_state=42)
)

scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
print(f"CV Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

## **📌 Заключение: Золотые правила валидации**

1. **Тестовый набор — священный грааль**  
   Никогда не используйте тестовые данные для обучения или настройки
   
2. **Кросс-валидация — ваш друг**  
   Всегда используйте CV для небольших/средних датасетов
   
3. **Учитывайте структуру данных**  
   Для временных рядов, групповых данных и несбалансированных классов используйте специализированные методы
   
4. **Автоматизируйте предобработку**  
   Всегда используйте Pipeline для предотвращения утечек данных

> "Правильное разделение данных — это не техническая деталь, а фундаментальный аспект построения надежных ML-моделей"
